# 📅 PyKwant Tutorial: Dates and Calendars

This notebook introduces the `pykwant.dates` module.

In Quantitative Finance, correctly handling dates is critical. A "year" is rarely exactly 365 days, and cash flows must roll to business days according to specific market conventions.

`pykwant` treats date logic functionally:

* **Calendars** are immutable sets of data.

* **Conventions** (Day Counts, Rolling Rules) are pure functions.

## 1. Setup and Imports

In [15]:
from datetime import date
from pykwant import dates

# Helper for visualization
def print_date_info(label: str, d: date):
    print(f"{label:<25}: {d} ({d.strftime('%A')})")

## 2. Module Documentation

Let's check the built-in documentation to see the available tools.

In [16]:
print(dates.__doc__)


Dates Module

This module handles date arithmetic, calendar management, and day count conventions
following a functional programming paradigm.

It provides:
- Immutable `Calendar` data structures.
- Pure functions for business day adjustments (Rolling Conventions).
- Standard Day Count Conventions (ACT/365, 30/360, etc.).
- Schedule generation for financial instruments.

All functions in this module are pure: they do not modify their inputs and
always return new date objects or values.



## 2. Creating a Calendar

A `Calendar` defines valid business days. It consists of a set of holidays and a definition of weekends. In `pykwant`, calendars are immutable `dataclasses`.

We will create a custom "Milan" calendar.

In [17]:
# Define holidays using frozenset (immutable)
holidays = frozenset([
    date(2025, 1, 1),   # New Year's Day
    date(2025, 1, 6),   # Epiphany
    date(2025, 4, 25),  # Liberation Day
    date(2025, 5, 1),   # Labor Day
    date(2025, 12, 25), # Christmas
    date(2025, 12, 26)  # St. Stephen's Day
])

# Create the Calendar
# We can use the Weekday enum for clarity
cal_milano = dates.Calendar(
    holidays=holidays,
    weekends=(dates.Weekday.SATURDAY, dates.Weekday.SUNDAY)
)

print(f"Calendar created with {len(cal_milano.holidays)} holidays.")

Calendar created with 6 holidays.


## 4. Checking Business Days

The function `is_business_day` checks if a date is a valid trading day (neither a weekend nor a holiday).

In [18]:
test_dates = [
    date(2025, 1, 1),  # Holiday (Wed)
    date(2025, 1, 4),  # Weekend (Sat)
    date(2025, 1, 7),  # Business Day (Tue)
]

print("--- Business Day Check ---")
for d in test_dates:
    is_biz = dates.is_business_day(d, cal_milano)
    status = "✅ Business Day" if is_biz else "❌ Non-Business Day"
    print_date_info(status, d)

--- Business Day Check ---
❌ Non-Business Day       : 2025-01-01 (Wednesday)
❌ Non-Business Day       : 2025-01-04 (Saturday)
✅ Business Day           : 2025-01-07 (Tuesday)


## 5. Rolling Conventions

When a payment falls on a non-business day, it must be adjusted. `pykwant` provides standard "Rolling Conventions" as pure functions.

* **Following**: Move to the next business day.

* **Preceding**: Move to the previous business day.

* **Modified Following**: Move forward, unless it crosses into the next month, then move back

In [19]:
# Saturday, January 4th
target_date = date(2025, 1, 4)

print("\n--- Rolling Conventions ---")
print_date_info("Original Date", target_date)

# 1. Following
d_foll = dates.following(target_date, cal_milano)
print_date_info("Following", d_foll)

# 2. Preceding
d_prec = dates.preceding(target_date, cal_milano)
print_date_info("Preceding", d_prec)

# 3. Modified Following (Critical for Month-End)
# Example: May 31st (Saturday). Following would be June 2nd.
# Modified Following must stay in May -> May 30th (Friday).
d_end_month = date(2025, 5, 31)
d_mf = dates.modified_following(d_end_month, cal_milano)

print("\n--- Modified Following Example ---")
print_date_info("End of Month (Sat)", d_end_month)
print_date_info("Adjusted Date", d_mf)


--- Rolling Conventions ---
Original Date            : 2025-01-04 (Saturday)
Following                : 2025-01-07 (Tuesday)
Preceding                : 2025-01-03 (Friday)

--- Modified Following Example ---
End of Month (Sat)       : 2025-05-31 (Saturday)
Adjusted Date            : 2025-05-30 (Friday)


## 6. Day Count Conventions

How much of a year is the period between two dates? This depends on the convention.

* **ACT/365**: Actual days / 365.

* **30/360**: Assumes 30-day months (standard for US Corp Bonds).

In [20]:
start = date(2025, 1, 1)
end = date(2025, 4, 1) # 90 days actually

print(f"\n--- Day Count Conventions ({start} -> {end}) ---")

dcc_act = dates.act_365(start, end)
dcc_30360 = dates.thirty_360(start, end)

print(f"ACT/365 Fraction: {dcc_act:.6f}")
print(f"30/360 Fraction:  {dcc_30360:.6f}")


--- Day Count Conventions (2025-01-01 -> 2025-04-01) ---
ACT/365 Fraction: 0.246575
30/360 Fraction:  0.250000


## 7. Schedule Generation

Finally, we can generate a complete schedule of payment dates. This function combines date arithmetic with rolling conventions.

In [21]:
print("\n--- Generated Schedule (Quarterly) ---")

schedule = dates.generate_schedule(
    start=date(2025, 1, 1),
    end=date(2026, 1, 1),
    freq_month=3,
    cal=cal_milano,
    convention=dates.modified_following
)

for i, d in enumerate(schedule):
    print_date_info(f"Payment {i+1}", d)


--- Generated Schedule (Quarterly) ---
Payment 1                : 2025-04-01 (Tuesday)
Payment 2                : 2025-07-01 (Tuesday)
Payment 3                : 2025-10-01 (Wednesday)
Payment 4                : 2026-01-01 (Thursday)
